In [ ]:
import os
print(f"현재 PID: {os.getpid()}")

In [7]:
# ResNet34_3ch + CBAM + 데이터 증강
# Resize / RandomHorizontalFlip / RandomRotation / RandomErasing

import os, re, numpy as np, torch, gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 및 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"
batch_size = 16
num_epochs = 10
learning_rate = 1e-4

# 데이터 증강
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])
val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 라벨 추출 함수
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# 데이터셋 정의
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)
        img = np.repeat(img, 3, axis=-1)    # (H, W, 3)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        return img, torch.tensor(label).long()

    def __len__(self):
        return len(self.paths)

# -------------------- CBAM --------------------
class ChannelAttention(nn.Module):
    def __init__(self, planes, ratio=16):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.max = nn.AdaptiveMaxPool2d(1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))

class SpatialAttention(nn.Module):
    def __init__(self, k=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = torch.mean(x, dim=1, keepdim=True)
        _max = torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None

    def forward(self, x):
        x = self.ca(x) * x
        attn = self.sa(x)
        self.last_attention = attn
        return attn * x

# -------------------- BasicBlock + CBAM --------------------
class BasicBlockCBAM(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.cbam = CBAM(planes) if use_cbam else None
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.cbam:
            out = self.cbam(out)

        if self.downsample:
            identity = self.downsample(x)

        out += identity
        return self.relu(out)

# -------------------- ResNet34 with CBAM --------------------
class ResNet34_CBAM(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 3)
        self.layer2 = self._make_layer(128, 4, stride=2)
        self.layer3 = self._make_layer(256, 6, stride=2)
        self.layer4 = self._make_layer(512, 3, stride=2, use_cbam=False)  # CBAM 생략

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 1)  # 그대로 두고

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.in_planes != planes * BasicBlockCBAM.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes * BasicBlockCBAM.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * BasicBlockCBAM.expansion))

        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam)]
        self.in_planes = planes * BasicBlockCBAM.expansion
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        return self.fc(torch.flatten(x, 1))

# 학습 루프
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42, stratify=labels)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    model = ResNet34_CBAM().to(device)
    
    criterion = nn.BCEWithLogitsLoss().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "best_model_resnet34_3ch_2.pth")

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device) 

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            probs = torch.sigmoid(outputs)         # [B, 1]
            preds = (probs > 0.5).long().squeeze(1)          # [B, 1] → binary prediction

            correct += (preds == labels.squeeze(1).long()).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        torch.cuda.empty_cache(); gc.collect()

        model.eval()
        correct = 0; total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.float().unsqueeze(1).to(device)

                outputs = model(images)
                
                probs = torch.sigmoid(outputs)         # [B, 1]
                preds = (probs > 0.5).long().squeeze(1)          # [B, 1] → binary prediction

                correct += (preds == labels.long()).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)

            probs = torch.sigmoid(outputs) 
            preds = (probs > 0.5).long().squeeze(1)

            y_probs.extend(probs.cpu().numpy().flatten())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy().flatten())

    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()

Using device: cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:14<00:00, 15.80it/s]


Train Acc: 66.0236, Loss: 0.6292
Val Acc: 10.5575
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:21<00:00, 11.06it/s]


Train Acc: 68.2744, Loss: 0.6067
Val Acc: 10.3325


...

Train Acc: 76.8489, Loss: 0.4859
Val Acc: 9.8100


[Epoch 9]: 100%|██████████| 234/234 [00:21<00:00, 11.10it/s]


Train Acc: 77.7063, Loss: 0.4682
Val Acc: 9.8950


[Epoch 10]: 100%|██████████| 234/234 [00:20<00:00, 11.58it/s]


Train Acc: 79.2069, Loss: 0.4461
Val Acc: 9.4825

📊 Test Evaluation:
✅ Test Accuracy: 67.00%
              precision    recall  f1-score   support

         0.0     0.6667    0.0075    0.0149       265
         1.0     0.6700    0.9981    0.8018       535

    accuracy                         0.6700       800
   macro avg     0.6683    0.5028    0.4084       800
weighted avg     0.6689    0.6700    0.5411       800

AUC: 0.6604
Confusion Matrix:
[[  2 263]
 [  1 534]]


In [ ]:
# ResNet34_3ch + 데이터 증강 (resize)

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from glob import glob
from tqdm import tqdm
import gc
from torchvision.models import resnet34, ResNet34_Weights
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay


# 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 로딩 및 3-way split
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

# 3채널 Dataset
class LIDC3ChannelDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        center_path = self.file_paths[idx]
        label = self.labels[idx]
        folder = os.path.dirname(center_path)
        fname = os.path.basename(center_path)
        slice_num = int(fname.split("_")[1])
        suffix = fname.split("_")[-1]

        slice_indices = [slice_num - 1, slice_num, slice_num + 1]
        images = []
        for sn in slice_indices:
            path = os.path.join(folder, f"slice_{sn:03d}_{suffix}")
            if os.path.exists(path):
                img = np.load(path).astype(np.float32)
            else:
                img = np.load(center_path).astype(np.float32)
            img = (img - img.min()) / (img.max() - img.min() + 1e-8)
            images.append(img)

        stacked = np.stack(images, axis=0)
        img_tensor = torch.tensor(stacked)
        img_tensor = F.interpolate(img_tensor.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
        if self.transform:
            img_tensor = self.transform(img_tensor)
        return img_tensor, torch.tensor(label).float(), center_path

    def __len__(self):
        return len(self.file_paths)

# Transform 정의
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Dataloader 구성
train_dataset = LIDC3ChannelDataset(train_files, train_labels, transform=transform)
val_dataset = LIDC3ChannelDataset(val_files, val_labels, transform=transform)
test_dataset = LIDC3ChannelDataset(test_files, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 모델 정의
model = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(DEVICE)

# 학습 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "best_model_resnet34_3ch.pth")
os.makedirs(os.path.dirname(save_path), exist_ok=True)
best_val_acc = 0.0

# 학습 루프
for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0.0
    correct, total = 0, 0  # ← 정확도용 초기화

    for images, labels, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images = images.to(DEVICE)
        labels = labels.unsqueeze(1).to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # 🎯 정확도 계산
        preds = (torch.sigmoid(outputs) > 0.5).long()
        correct += (preds == labels.long()).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"[Epoch {epoch+1}] Loss: {epoch_loss / len(train_loader):.4f}, Train Acc: {train_acc:.4f}")

    gc.collect()
    torch.cuda.empty_cache()

    # Validation
    model.eval()
    correct, total = 0, 0
    val_preds, val_labels_ = [], []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(images).squeeze()

            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)
            val_preds.extend(preds.cpu().numpy())
            val_labels_.extend(labels.cpu().numpy())

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved!")


# --- 테스트 결과 출력 ---
print("\n\U0001F4CA Test Set Evaluation (Best Model \uae30\uc900):")
model.load_state_dict(torch.load(save_path))
model.eval()

y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels, _ in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).long()

        y_probs.extend(probs.cpu().numpy())
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Accuracy
test_acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"\u2705 Test Accuracy: {test_acc:.2f}%")

# Classification Report
print(classification_report(y_true, y_pred, digits=4))

# AUC 출력
try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC \uacc4\uc0b0 \uc2e4\ud328: \uc591/\uc74c \ud074\ub798\uc2a4\uac00 \ubaa8\ub450 \uc788\uc5b4\uc57c \ud568.")

# Confusion Matrix 출력
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)


Epoch 1/100: 100%|██████████| 234/234 [00:25<00:00,  9.28it/s]


[Epoch 1] Loss: 0.6154
Validation Accuracy: 0.7063
✅ Best model saved!


Epoch 2/100: 100%|██████████| 234/234 [00:23<00:00,  9.91it/s]


[Epoch 2] Loss: 0.4915
Validation Accuracy: 0.6837


Epoch 3/100: 100%|██████████| 234/234 [00:25<00:00,  9.25it/s]


...

Epoch 99/100: 100%|██████████| 234/234 [00:23<00:00,  9.94it/s]


[Epoch 99] Loss: 0.0169
Validation Accuracy: 0.8875


Epoch 100/100: 100%|██████████| 234/234 [00:23<00:00, 10.00it/s]


[Epoch 100] Loss: 0.0205
Validation Accuracy: 0.8738

📊 Test Set Evaluation (Best Model 기준):
✅ Test Accuracy: 89.88%
              precision    recall  f1-score   support

         0.0     0.8819    0.8145    0.8469       275
         1.0     0.9066    0.9429    0.9244       525

    accuracy                         0.8988       800
   macro avg     0.8942    0.8787    0.8856       800
weighted avg     0.8981    0.8988    0.8977       800

AUC: 0.9572
Confusion Matrix:
[[224  51]
 [ 30 495]]


: 